In [7]:
# ------ 1. 데이터 전처리 ------
# 데이터 링크 : https://www.kaggle.com/aashita/nyt-comments
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [8]:
df['headline'].isnull().sum()

0

In [9]:
headline = df['headline'].tolist()
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [10]:
print('총 헤드라인 개수 : ', len(headline))
headline = [h for h in headline if h != 'Unknown']
print('Unknown 제거 후 헤드라인 개수: ', len(headline))

총 헤드라인 개수 :  1324
Unknown 제거 후 헤드라인 개수:  1214


In [11]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [18]:
# 구두점 제거, 소문자화
def reprocessing(s):
    s = re.sub(r'[^\w ]', '', s).lower()
    return s

text = [reprocessing(h) for h in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [19]:
# 단어 집합의 크기
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: ', vocab_size)


단어 집합의 크기:  3494


In [28]:
sequences = []

for line in text:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]


[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [30]:
# 인덱스로부터 단어를 찾는 딕셔너리 생성
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key
    
print(index_to_word[831])


disaster


In [31]:
# 패딩 작업위해 가장 긴 샘플의 길이 확인
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이: ', max_len)


샘플의 최대 길이:  24


In [34]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [35]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [36]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [37]:
print(y[:3])

[ 269  371 1115]


In [38]:
# 레이블 y를 원-핫 인코딩
y = to_categorical(y)

In [40]:
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [42]:
# ------ 2. 모델 설계하기 ------
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6293 - accuracy: 0.0260
Epoch 2/200
244/244 - 4s - loss: 7.1073 - accuracy: 0.0301
Epoch 3/200
244/244 - 4s - loss: 6.9658 - accuracy: 0.0342
Epoch 4/200
244/244 - 4s - loss: 6.8324 - accuracy: 0.0411
Epoch 5/200
244/244 - 4s - loss: 6.6684 - accuracy: 0.0443
Epoch 6/200
244/244 - 4s - loss: 6.4976 - accuracy: 0.0502
Epoch 7/200
244/244 - 4s - loss: 6.3061 - accuracy: 0.0552
Epoch 8/200
244/244 - 4s - loss: 6.1078 - accuracy: 0.0607
Epoch 9/200
244/244 - 4s - loss: 5.9139 - accuracy: 0.0638
Epoch 10/200
244/244 - 4s - loss: 5.7336 - accuracy: 0.0654
Epoch 11/200
244/244 - 4s - loss: 5.5599 - accuracy: 0.0720
Epoch 12/200
244/244 - 4s - loss: 5.3996 - accuracy: 0.0742
Epoch 13/200
244/244 - 4s - loss: 5.2418 - accuracy: 0.0824
Epoch 14/200
244/244 - 4s - loss: 5.0982 - accuracy: 0.0920
Epoch 15/200
244/244 - 4s - loss: 4.9560 - accuracy: 0.1002
Epoch 16/200
244/244 - 4s - loss: 4.8182 - accuracy: 0.1116
Epoch 17/200
244/244 - 4s - loss: 4.6877 - accura

244/244 - 4s - loss: 0.3412 - accuracy: 0.9150
Epoch 138/200
244/244 - 4s - loss: 0.3440 - accuracy: 0.9127
Epoch 139/200
244/244 - 4s - loss: 0.3386 - accuracy: 0.9152
Epoch 140/200
244/244 - 4s - loss: 0.3324 - accuracy: 0.9154
Epoch 141/200
244/244 - 4s - loss: 0.3277 - accuracy: 0.9136
Epoch 142/200
244/244 - 4s - loss: 0.3248 - accuracy: 0.9161
Epoch 143/200
244/244 - 4s - loss: 0.3247 - accuracy: 0.9146
Epoch 144/200
244/244 - 4s - loss: 0.3203 - accuracy: 0.9155
Epoch 145/200
244/244 - 4s - loss: 0.3192 - accuracy: 0.9159
Epoch 146/200
244/244 - 4s - loss: 0.3161 - accuracy: 0.9155
Epoch 147/200
244/244 - 4s - loss: 0.3126 - accuracy: 0.9146
Epoch 148/200
244/244 - 4s - loss: 0.3101 - accuracy: 0.9145
Epoch 149/200
244/244 - 4s - loss: 0.3080 - accuracy: 0.9164
Epoch 150/200
244/244 - 4s - loss: 0.3059 - accuracy: 0.9157
Epoch 151/200
244/244 - 4s - loss: 0.3046 - accuracy: 0.9158
Epoch 152/200
244/244 - 4s - loss: 0.3113 - accuracy: 0.9153
Epoch 153/200
244/244 - 4s - loss: 0.3

In [56]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            if index == result:
                current_word = current_word + ' ' + word
                break
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [71]:
sentence_generation(model, t, 'trump', 10)

'trump returns to a hard line on immigrants as how a'